测试 albert_small 情感分类  
https://github.com/bojone/bert4keras/blob/master/examples/task_sentiment_albert.py

# part1:配置环境 + 加载文件

In [1]:
#! pip install keras_bert
! pip install git+https://www.github.com/bojone/bert4keras.git
import tensorflow as tf
from tensorflow.python.client import device_lib
print(tf.__version__)
tf.test.gpu_device_name()
device_lib.list_local_devices()
!nvidia-smi

  Cloning https://www.github.com/bojone/bert4keras.git to /tmp/pip-req-build-wtazt4l8
  Running command git clone -q https://www.github.com/bojone/bert4keras.git /tmp/pip-req-build-wtazt4l8
  Created wheel for bert4keras: filename=bert4keras-0.8.3-cp36-none-any.whl size=41795 sha256=403804f3ee799ba5e6d95a02859c3bd0ef8bca1b52ab64e562968710f1beb10d
  Stored in directory: /tmp/pip-ephem-wheel-cache-o2rq9ix5/wheels/12/58/83/8ff5c864b80c860e6d9e9e0d90c04fafca05d01d21f9f6fcba
Successfully built bert4keras
2.2.0
Tue Jul 21 12:00:45 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      | 

### 加载数据文件

In [2]:
! git clone https://github.com/bojone/bert4keras.git
! unzip -d '/content/sample_data' "/content/bert4keras/examples/datasets/sentiment.zip"

Cloning into 'bert4keras'...
remote: Enumerating objects: 5684, done.
remote: Total 5684 (delta 0), reused 0 (delta 0), pack-reused 5684
Receiving objects: 100% (5684/5684), 9.20 MiB | 23.27 MiB/s, done.
Resolving deltas: 100% (3741/3741), done.
Archive:  /content/bert4keras/examples/datasets/sentiment.zip
   creating: /content/sample_data/sentiment/
  inflating: /content/sample_data/sentiment/sentiment.test.data  
  inflating: /content/sample_data/sentiment/sentiment.train.data  
  inflating: /content/sample_data/sentiment/sentiment.valid.data  


#### 1.1 加载ALbert_small

In [ ]:
! git clone https://github.com/bojone/bert4keras.git
! unzip -d '/content/sample_data' "/content/bert4keras/examples/datasets/sentiment.zip"

#### 1.2 Albert_large_zh

该版本不能被直接使用需要转换  
https://github.com/bojone/albert_zh

In [ ]:
## 百度下载的 albert_large
! unzip -o "/content/drive/My Drive/local_copy/albert_large_google_zh.zip" -d '/content/albert_large_google_zh'

#### 1.3 Chinese-BERT-wwm


In [3]:
# 加载drive中的bert checkpoint
#! wget https://github.com/ymcui/Chinese-BERT-wwm
! unzip "/content/drive/My Drive/Colab Notebooks/chinese_wwm_L-12_H-768_A-12.zip" 

Archive:  /content/drive/My Drive/Colab Notebooks/chinese_wwm_L-12_H-768_A-12.zip
   creating: publish/
  inflating: publish/vocab.txt       
  inflating: publish/bert_model.ckpt.index  
  inflating: publish/bert_model.ckpt.data-00000-of-00001  
  inflating: publish/bert_config.json  
  inflating: publish/bert_model.ckpt.meta  


# part2 配置全局参数

In [27]:
import json
from tqdm import tqdm
import os, re
import numpy as np
import pandas as pd
import codecs

import numpy as np
import keras
from bert4keras.backend import keras, set_gelu
from bert4keras.tokenizers import Tokenizer
from bert4keras.models import build_transformer_model
from bert4keras.optimizers import Adam, extend_with_piecewise_linear_lr
from bert4keras.snippets import sequence_padding, DataGenerator
from bert4keras.snippets import open
from keras.layers import Lambda, Dense
from bert4keras.optimizers import *

set_gelu('tanh')  # 切换gelu版本

num_classes = 2
maxlen = 128
batch_size = 64
# 1 训练数据文件路径
data_dir = "/content/sample_data/sentiment"
train_data_dir = data_dir + "/sentiment.train.data"
valid_data_dir = data_dir + "/sentiment.valid.data"
test_data_dir = data_dir + "/sentiment.test.data"

print(os.path.exists(train_data_dir))

# 3 训练模型文件路径
checkpoint_dir = 'checkpoint/subject_extract2'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
# 4 结果输出路径    
outputdir = '/content/drive/My Drive'

True


### 2.1 albert_small_google

In [ ]:

config_path = '/content/albert_small_zh_google/albert_config_small_google.json'
checkpoint_path = '/content/albert_small_zh_google/albert_model.ckpt'
dict_path = '/content/albert_small_zh_google/vocab.txt'


### 2.2 albert_large_google

In [ ]:
model_dir = "/content/albert_large_google_zh/albert_large_google_zh"
config_path = model_dir + '/albert_config.json'
checkpoint_path = model_dir + '/albert_model.ckpt'
dict_path = model_dir + '/vocab.txt'

### 2.3 bert_base

In [18]:
config_path = './publish/bert_config.json'
checkpoint_path = './publish/bert_model.ckpt'
dict_path = './publish/vocab.txt'
 
print('load checkpoint path : ',os.path.exists(config_path))

load checkpoint path :  True


# part 3 模型部分

### 数据构建

In [15]:

def load_data(filename):
    D = []
    with open(filename, encoding='utf-8') as f:
        for l in f:
            text, label = l.strip().split('\t')
            D.append((text, int(label)))
    return D


# 加载数据集
train_data = load_data(train_data_dir)
valid_data = load_data(valid_data_dir)
test_data = load_data(test_data_dir)

# 建立分词器
tokenizer = Tokenizer(dict_path, do_lower_case=True)

class data_generator(DataGenerator):
    """数据生成器
    """
    def __iter__(self, random=False):
        batch_token_ids, batch_segment_ids, batch_labels = [], [], []
        for is_end, (text, label) in self.sample(random):
            token_ids, segment_ids = tokenizer.encode(text, maxlen=maxlen)
            batch_token_ids.append(token_ids)
            batch_segment_ids.append(segment_ids)
            batch_labels.append([label])
            if len(batch_token_ids) == self.batch_size or is_end:
                batch_token_ids = sequence_padding(batch_token_ids)
                batch_segment_ids = sequence_padding(batch_segment_ids)
                batch_labels = sequence_padding(batch_labels)
                yield [batch_token_ids, batch_segment_ids], batch_labels
                batch_token_ids, batch_segment_ids, batch_labels = [], [], []

# 转换数据集
train_generator = data_generator(train_data, batch_size)
valid_generator = data_generator(valid_data, batch_size)
test_generator = data_generator(test_data, batch_size)


In [16]:
# 查看数据
tag = 0
for i in train_generator.forfit():
  tag +=1
  if tag>1:
    break
  #print(i,len(i))
  #print(i[0],'\n',i[1])
  #print(i[0][0],'\n',i[0][0].shape)
  #print(i[0][1],'\n',i[0][1].shape)
  #print(i[1],'\n',i[1].shape)
  print('ids : {} \ntext:{}\nlabel:{}'.format(i[0][0][0],tokenizer.decode(i[0][0][0]),i[1][0]))

ids : [ 101 5885 4281 1920 3362 5108 8013 3362 3171 2682 6629 7770  676 1939
 3159 4638 6929 3667 3189 2094 8080  102    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0] 
text:蒙牛大果粒！果断想起高三奋斗的那段日子～
label:[1]


###  模型构建

## ALBERT

In [ ]:

# 加载预训练模型
## return_keras_model=False 不训练层
bert = build_transformer_model(
    config_path=config_path,
    checkpoint_path=checkpoint_path,
    model='albert',
    return_keras_model=False,
)

output = Lambda(lambda x: x[:, 0], name='CLS-token')(bert.model.output)#只取cls值
output = Dense(
    units=num_classes,
    activation='softmax',
    kernel_initializer=bert.initializer
)(output)

model = keras.models.Model(bert.model.input, output)
model.summary()

# 派生为带分段线性学习率的优化器。
# 其中name参数可选，但最好填入，以区分不同的派生优化器。


### Bert

In [32]:
bert = build_transformer_model(
    config_path=config_path,
    checkpoint_path=checkpoint_path,
    return_keras_model=True,
)
layer_name = 'Transformer-3-FeedForward-Norm'                                    


output = Lambda(lambda x: x[:, 0], name='CLS-token')(bert.get_layer(layer_name).output)#只取cls值
output = Dense(
    units=num_classes,
    activation='softmax',
    kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.02)
)(output)

model = keras.models.Model(bert.input, output)
model.summary()

Model: "model_14"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, None)         0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Embedding-Token (Embedding)     (None, None, 768)    16226304    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, None, 768)    1536        Input-Segment[0][0]              
___________________________________________________________________________________________

### optimizer

#### 3.1 线性Adam

In [33]:
model.compile(
    loss='sparse_categorical_crossentropy',
    # optimizer=Adam(1e-5),  # 用足够小的学习率
    optimizer=Adam(learning_rate=1e-4),
    metrics=['accuracy'],
)

#### 3.2 线性AdamLR
可以分段调控 ，线性变化

In [ ]:
AdamLR = extend_with_piecewise_linear_lr(Adam, name='AdamLR')

model.compile(
    loss='sparse_categorical_crossentropy',
    # optimizer=Adam(1e-5),  # 用足够小的学习率
    optimizer=AdamLR(learning_rate=1e-4, lr_schedule={
        1000: 1,
        2000: 0.1
    }),
    metrics=['accuracy'],
)

#### 3.3 线性AdaFactor
这个优化器lr batch_size 需要大一点

In [28]:
model.compile(
    loss='sparse_categorical_crossentropy',
    # optimizer=Adam(1e-3),  # 用足够小的学习率
    optimizer=AdaFactor(learning_rate=1e-3),
    metrics=['accuracy'],
)

evaluator = Evaluator()

model.fit(
    train_generator.forfit(),
    steps_per_epoch=len(train_generator),
    epochs=5,
    callbacks=[evaluator]
)

model.load_weights('best_model.weights')
print(u'final test acc: %05f\n' % (evaluate(test_generator)))

### callback

In [29]:

def evaluate(data):
    total, right = 0., 0.
    for x_true, y_true in data:
        y_pred = model.predict(x_true).argmax(axis=1)
        y_true = y_true[:, 0]
        total += len(y_true)
        right += (y_true == y_pred).sum()
    return right / total


class Evaluator(keras.callbacks.Callback):
    def __init__(self):
        self.best_val_acc = 0.

    def on_epoch_end(self, epoch, logs=None):
        val_acc = evaluate(valid_generator)
        if val_acc > self.best_val_acc:
            self.best_val_acc = val_acc
            model.save_weights('best_model.weights')
        test_acc = evaluate(test_generator)
        print(
            u'val_acc: %.5f, best_val_acc: %.5f, test_acc: %.5f\n' %
            (val_acc, self.best_val_acc, test_acc)
        )



### train

In [ ]:

evaluator = Evaluator()

model.fit(
    train_generator.forfit(),
    steps_per_epoch=len(train_generator),
    epochs=10,
    callbacks=[evaluator]
)

model.load_weights('best_model.weights')
print(u'final test acc: %05f\n' % (evaluate(test_generator)))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
264/264 [==============================] - 257s 973ms/step - loss: 0.2609 - accuracy: 0.8939
val_acc: 0.92373, best_val_acc: 0.92373, test_acc: 0.92610

Epoch 2/10
264/264 [==============================] - 255s 964ms/step - loss: 0.1196 - accuracy: 0.9569
val_acc: 0.93842, best_val_acc: 0.93842, test_acc: 0.94694

Epoch 3/10
264/264 [==============================] - 255s 964ms/step - loss: 0.0604 - accuracy: 0.9787
val_acc: 0.92563, best_val_acc: 0.93842, test_acc: 0.93036

Epoch 4/10
264/264 [==============================] - 255s 965ms/step - loss: 0.0376 - accuracy: 0.9873
val_acc: 0.94363, best_val_acc: 0.94363, test_acc: 0.94410

Epoch 5/10
264/264 [==============================] - 255s 964ms/step - loss: 0.0271 - accuracy: 0.9909
val_acc: 0.93558, best_val_acc: 0.94363, test_acc: 0.94173

Epoch 6/10
264/264 [==============================] - 255s 965ms/step - loss: 0.0219 - accuracy: 0.9922
val_acc: 0.94315, best_val_acc: 0.94363, test_acc: 0.93984

Epoch 7/10
264/2

# part 4 错误分析

In [ ]:
# 打印错误数
num = 6
tag = 0
for x_true, y_true in test_generator:
    tag +=1
    if tag>num:
      break
    #print(x_true[0].shape)
    y_pred = model.predict(x_true).argmax(axis=1)
    y_true = y_true[:, 0]
    #print(y_true)
    #print(y_pred)
    #error_pred = x_true[0][y_true!=y_pred]
    error_ids = np.array(list(range(len(y_true))))[y_true!=y_pred]

    print(error_ids)
    for ids in error_ids:
      print('text :{}\npredict:{}, ture:{}\n'.format(tokenizer.decode(x_true[0][ids]),y_pred[ids],y_true[ids]))